In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

### task1

In [ ]:
df = pd.read_csv("/content/House_Price.csv")
df.head()

,price,crime_rate,resid_area,air_qual,room_num,age,dist1,dist2,dist3,dist4,teachers,poor_prop,airport,n_hos_beds,n_hot_rooms,waterbody,rainfall,bus_ter,parks
0,24.0,0.00632,32.31,0.538,6.575,65.2,4.35,3.81,4.18,4.01,24.7,4.98,YES,5.480,11.1920,River,23,YES,0.049347
1,21.6,0.02731,37.07,0.469,6.421,78.9,4.99,4.70,5.12,5.06,22.2,9.14,NO,7.332,12.1728,Lake,42,YES,0.046146
2,34.7,0.02729,37.07,0.469,7.185,61.1,5.03,4.86,5.01,4.97,22.2,4.03,NO,7.394,101.1200,NaN,38,YES,0.045764
3,33.4,0.03237,32.18,0.458,6.998,45.8,6.21,5.93,6.16,5.96,21.3,2.94,YES,9.268,11.2672,Lake,45,YES,0.047151
4,36.2,0.06905,32.18,0.458,7.147,54.2,6.16,5.86,6.37,5.86,21.3,5.33,NO,8.824,11.2896,Lake,55,YES,0.039474


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   price        506 non-null    float64
 1   crime_rate   506 non-null    float64
 2   resid_area   506 non-null    float64
 3   air_qual     506 non-null    float64
 4   room_num     506 non-null    float64
 5   age          506 non-null    float64
 6   dist1        506 non-null    float64
 7   dist2        506 non-null    float64
 8   dist3        506 non-null    float64
 9   dist4        506 non-null    float64
 10  teachers     506 non-null    float64
 11  poor_prop    506 non-null    float64
 12  airport      506 non-null    object 
 13  n_hos_beds   498 non-null    float64
 14  n_hot_rooms  506 non-null    float64
 15  waterbody    351 non-null    object 
 16  rainfall     506 non-null    int64  
 17  bus_ter      506 non-null    object 
 18  parks        506 non-null    float64
dtypes: float

#### 1.Исключите сильно коррелированные независимые переменные. Под
 ##### ***Справка 1:***
 *слишком высокой корреляцией можно понимать абсолютное значение коэффициента корреляции выше 0.9. Это поможет избежать проблемы мультиколлинеарности в модели.*

 #### ***Справка 2:***
**Мультиколлинеарность** *в линейной регрессии — это явление, при котором одна из входных переменных линейно зависит от других входных переменных, то есть между ними наблюдается сильная корреляция.*


In [ ]:
def preprocessing_df(df, threshold=0.9):
    """
    Функция preprocessing_df предназначена для удаления сильно
    коррелированных признаков из DataFrame на основе порогового значения корреляции.
    Предварительно происходит обработка пропущенных значений (метод Mean Imputation).
    """

    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())


    categorical_cols = df.select_dtypes(include=[object]).columns
    df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])


    corr_matrix = df.corr(numeric_only=True)
    triangle_corr_matrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    to_drop_column = [column for column in triangle_corr_matrix.columns if any(abs(triangle_corr_matrix[column]) > threshold)]

    df = df.drop(columns=to_drop_column)

    print(f"Сильно коррелированные переменные: {to_drop_column}")
    return df


In [ ]:
df = preprocessing_df(df)
df.head()

Сильно коррелированные переменные: ['dist2', 'dist3', 'dist4', 'parks']


,price,crime_rate,resid_area,air_qual,room_num,age,dist1,teachers,poor_prop,airport,n_hos_beds,n_hot_rooms,waterbody,rainfall,bus_ter
0,24.0,0.00632,32.31,0.538,6.575,65.2,4.35,24.7,4.98,YES,5.480,11.1920,River,23,YES
1,21.6,0.02731,37.07,0.469,6.421,78.9,4.99,22.2,9.14,NO,7.332,12.1728,Lake,42,YES
2,34.7,0.02729,37.07,0.469,7.185,61.1,5.03,22.2,4.03,NO,7.394,101.1200,River,38,YES
3,33.4,0.03237,32.18,0.458,6.998,45.8,6.21,21.3,2.94,YES,9.268,11.2672,Lake,45,YES
4,36.2,0.06905,32.18,0.458,7.147,54.2,6.16,21.3,5.33,NO,8.824,11.2896,Lake,55,YES


####2.Преобразуйте категориальные переменные в числовой формат при помощи кодирования фиктивными переменными one-hot кодирование.


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   price        506 non-null    float64
 1   crime_rate   506 non-null    float64
 2   resid_area   506 non-null    float64
 3   air_qual     506 non-null    float64
 4   room_num     506 non-null    float64
 5   age          506 non-null    float64
 6   dist1        506 non-null    float64
 7   teachers     506 non-null    float64
 8   poor_prop    506 non-null    float64
 9   airport      506 non-null    object 
 10  n_hos_beds   506 non-null    float64
 11  n_hot_rooms  506 non-null    float64
 12  waterbody    506 non-null    object 
 13  rainfall     506 non-null    int64  
 14  bus_ter      506 non-null    object 
dtypes: float64(11), int64(1), object(3)
memory usage: 59.4+ KB


In [ ]:
assert (list(df['bus_ter'].unique()) == ['YES'])

In [ ]:
df = df.drop(columns = 'bus_ter')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   price        506 non-null    float64
 1   crime_rate   506 non-null    float64
 2   resid_area   506 non-null    float64
 3   air_qual     506 non-null    float64
 4   room_num     506 non-null    float64
 5   age          506 non-null    float64
 6   dist1        506 non-null    float64
 7   teachers     506 non-null    float64
 8   poor_prop    506 non-null    float64
 9   airport      506 non-null    object 
 10  n_hos_beds   506 non-null    float64
 11  n_hot_rooms  506 non-null    float64
 12  waterbody    506 non-null    object 
 13  rainfall     506 non-null    int64  
dtypes: float64(11), int64(1), object(2)
memory usage: 55.5+ KB


In [ ]:
def encode_categorial_var (data):
    """
    Функция encode_categorial_var предназначена для применения one_hot кодирования
    для column из DataFrame с типом Dtype=object.
    """
    categorial_cols = data.select_dtypes(include=[object]).columns
    data = pd.get_dummies(data, columns=categorial_cols, drop_first=True)
    bool_columns = data.select_dtypes(include=[bool]).columns
    data[bool_columns] = data[bool_columns].astype(int)

    return data

In [ ]:
df1 = encode_categorial_var(df)
df1.head()

,price,crime_rate,resid_area,air_qual,room_num,age,dist1,teachers,poor_prop,n_hos_beds,n_hot_rooms,rainfall,airport_YES,waterbody_Lake and River,waterbody_River
0,24.0,0.00632,32.31,0.538,6.575,65.2,4.35,24.7,4.98,5.480,11.1920,23,1,0,1
1,21.6,0.02731,37.07,0.469,6.421,78.9,4.99,22.2,9.14,7.332,12.1728,42,0,0,0
2,34.7,0.02729,37.07,0.469,7.185,61.1,5.03,22.2,4.03,7.394,101.1200,38,0,0,1
3,33.4,0.03237,32.18,0.458,6.998,45.8,6.21,21.3,2.94,9.268,11.2672,45,1,0,0
4,36.2,0.06905,32.18,0.458,7.147,54.2,6.16,21.3,5.33,8.824,11.2896,55,0,0,0


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   price                     506 non-null    float64
 1   crime_rate                506 non-null    float64
 2   resid_area                506 non-null    float64
 3   air_qual                  506 non-null    float64
 4   room_num                  506 non-null    float64
 5   age                       506 non-null    float64
 6   dist1                     506 non-null    float64
 7   teachers                  506 non-null    float64
 8   poor_prop                 506 non-null    float64
 9   n_hos_beds                506 non-null    float64
 10  n_hot_rooms               506 non-null    float64
 11  rainfall                  506 non-null    int64  
 12  airport_YES               506 non-null    int64  
 13  waterbody_Lake and River  506 non-null    int64  
 14  waterbody_

In [ ]:
df2 = df1.drop(columns = 'waterbody_Lake and River')

In [ ]:
df2.head()

,price,crime_rate,resid_area,air_qual,room_num,age,dist1,teachers,poor_prop,n_hos_beds,n_hot_rooms,rainfall,airport_YES,waterbody_River
0,24.0,0.00632,32.31,0.538,6.575,65.2,4.35,24.7,4.98,5.480,11.1920,23,1,1
1,21.6,0.02731,37.07,0.469,6.421,78.9,4.99,22.2,9.14,7.332,12.1728,42,0,0
2,34.7,0.02729,37.07,0.469,7.185,61.1,5.03,22.2,4.03,7.394,101.1200,38,0,1
3,33.4,0.03237,32.18,0.458,6.998,45.8,6.21,21.3,2.94,9.268,11.2672,45,1,0
4,36.2,0.06905,32.18,0.458,7.147,54.2,6.16,21.3,5.33,8.824,11.2896,55,0,0


#### 3.Проверьте, не улучшится ли связь некоторых признаков с зависимойпеременной при применении различных преобразований. (Например,логарифмирования или взятия квадратного корня).

*Если после преобразования переменной её корреляция с целевой переменной увеличивается, используйте преобразованную версию этой переменной.*

In [ ]:

def update_features(df, target_column):
    """
    Функция update_features предназначена для получения лучшего преобразования
    признаков (при применении которого, корреляция с целевой переменной увеличится).
    """

    transformed_features = {}

    for column in df.select_dtypes(include=[np.number]).columns:
        if column == target_column:
            continue
        else:
            original_corr = df[target_column].corr(df[column])

            log_transformed = np.log1p(df[column].clip(lower=1e-9))
            log_corr = df[target_column].corr(log_transformed)

            sqrt_transformed = np.sqrt(df[column].clip(lower=0))
            sqrt_corr = df[target_column].corr(sqrt_transformed)

            best_corr = original_corr
            best_transformed = None

            if abs(log_corr) > abs(best_corr):
                best_corr = log_corr
                best_transformed = log_transformed

            if abs(sqrt_corr) > abs(best_corr):
                best_corr = sqrt_corr
                best_transformed = sqrt_transformed

            if best_transformed is not None:
                transformed_features[column] = best_transformed
                print(f"Переменная {column} улучшена.")

    for col, transformed in transformed_features.items():
        df[col] = transformed

    return df


In [ ]:
df3 = update_features(df2,'price')

Переменная crime_rate улучшена.
Переменная resid_area улучшена.
Переменная air_qual улучшена.
Переменная dist1 улучшена.
Переменная teachers улучшена.
Переменная poor_prop улучшена.
Переменная rainfall улучшена.


<ipython-input-64-a42043295a5a>:15: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  log_transformed = np.log1p(df[column].clip(lower=1e-9))
<ipython-input-64-a42043295a5a>:15: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  log_transformed = np.log1p(df[column].clip(lower=1e-9))


In [ ]:
df3.head()

,price,crime_rate,resid_area,air_qual,room_num,age,dist1,teachers,poor_prop,n_hos_beds,n_hot_rooms,rainfall,airport_YES,waterbody_River
0,24.0,0.006300,3.505858,0.733485,6.575,65.2,1.677097,3.246491,1.788421,5.480,11.1920,3.178054,1,1
1,21.6,0.026944,3.639427,0.684836,6.421,78.9,1.790091,3.144152,2.316488,7.332,12.1728,3.761200,0,0
2,34.7,0.026924,3.639427,0.684836,7.185,61.1,1.796747,3.144152,1.615420,7.394,101.1200,3.663562,0,1
3,33.4,0.031857,3.501947,0.676757,6.998,45.8,1.975469,3.104587,1.371181,9.268,11.2672,3.828641,1,0
4,36.2,0.066770,3.501947,0.676757,7.147,54.2,1.968510,3.104587,1.845300,8.824,11.2896,4.025352,0,0


### task2

при выполнении ряда естественных условий, оптимальные
коэффициенты βˆ можно выразить как:
$$\hat{\boldsymbol{\beta}}=\left(\mathbf{X}^{T} \mathbf{X}\right)^{-1} \mathbf{X}^{T} \mathbf{y}$$

*где “крышечка” подчеркивает что это оценка, а не реальные β.*

После того, как мы нашли приближённые коэффициенты βˆ, мы можем
получить предсказания ˆy по x–ам из нашей выборки:


$$\hat{y}_{i}=\hat{\beta}_{0}+\hat{\beta}_{1} x_{i 1}+\hat{\beta}_{2} x_{i 2}+\cdots+\hat{\beta}_{k} x_{i k}$$

Эти yˆ-и содержат приближение линейной части нашей модели, зависящей от x-ов, но они не включают в себя отклонения от прочих факторов, представленные ошибками :
$$\tilde{\varepsilon}_{i}=y_{i}-\hat{y}_{i}$$



In [ ]:
def linear_regression(df, target_var, feautures_names):
    """
    Функция linear_regression предназначена для получения оценки соответсвующих
    параметров линейной регрессии.
    feautures_names - список строк с именами признаков
    """
    print('============================================')
    print(f" feautures: {feautures_names}")

    X = df[feautures_names].values
    Y = df[target_var].values

    X = np.hstack((np.ones((X.shape[0], 1)), X))



    estimates_of_b = np.linalg.inv((X.T @ X)) @ (X.T @ Y)

    estimates_of_y = X @ estimates_of_b

    estimates_of_errors = Y - estimates_of_y

    #https://dit.isuct.ru/IVT/BOOKS/Model/Model1/VECH/metod/metod9/part2.htm
    sigma_squared = np.sum(estimates_of_errors**2) / (X.shape[0] - X.shape[1])
    print("Оценка параметров линейной регрессии:")
    print(f"Intercept (β0): {estimates_of_b[0]}")
    print(f"Коэффициенты (β): {estimates_of_b[1:]}")
    print(f"Оценка дисперсии ошибок (σ̂²): {sigma_squared}")


    return estimates_of_b, estimates_of_y, sigma_squared


In [ ]:
estimates_of_b, estimates_of_y, estimates_of_errors, estimates_of_dis  = get_estimate(df2, 'price', df2.drop(columns=['price']).columns.tolist())

 feautures: ['crime_rate', 'resid_area', 'air_qual', 'room_num', 'age', 'dist1', 'teachers', 'poor_prop', 'n_hos_beds', 'n_hot_rooms', 'rainfall', 'airport_YES', 'waterbody_River']
Оценка параметров линейной регрессии:
Intercept (β0): 12.435468772366448
Коэффициенты (β): [-5.10547675e-01 -2.66277810e+00 -2.42479750e+01  2.76132002e+00
  1.64990274e-02 -7.87140876e+00  1.77297734e+01 -1.05669926e+01
  2.40286065e-01  2.03149389e-02  7.62971914e-02  1.04735364e+00
 -1.83115753e-01]
Оценка дисперсии ошибок (σ̂²): 18.68857595530486


### task3
1.  Найдите оценку дисперсии βˆs, используя вместо σ^2 оценку регрессионных ошибок из предыдущего задания;
2. Найдите стандартизацию коэффициента βˆs при условии выполнения основной гипотезы H0;
3. Предполагая, что стандартизация βˆs имеет приблизительно стандартное нормальное распределение и что более “экстремальными” (более указывающими на альтернативную гипотезу) значениями являются большие по модулю значения, найдите p–value для этого теста.



Полученные ранее оценки βˆ0, βˆ1, . . . , βˆk
будут являться нормальными случайными величинами, т.к. они получены как линейные комбинации y–ов = $\hat{\boldsymbol{\beta}}=\left(\mathbf{X}^{T} \mathbf{X}\right)^{-1} \mathbf{X}^{T} \mathbf{y}$

Их средние значения равны реальным параметрам β0, β1, . . . , βk. А их диспер-
сии можно получить из матрицы ковариации βˆ:

$$
\left(\begin{array}{ccc}
\mathbb{D}\left(\hat{\beta}_{0}\right) & \operatorname{Cov}\left(\hat{\beta}_{0}, \hat{\beta}_{1}\right) & \cdots & \operatorname{Cov}\left(\hat{\beta}_{0}, \hat{\beta}_{k}\right) \\
\operatorname{Cov}\left(\hat{\beta}_{1}, \hat{\beta}_{0}\right) & \mathbb{D}\left(\hat{\beta}_{1}\right) & \cdots & \operatorname{Cov}\left(\hat{\beta}_{1}, \hat{\beta}_{k}\right) \\
\vdots & \vdots & \ddots & \vdots \\
\operatorname{Cov}\left(\hat{\beta}_{k}, \hat{\beta}_{0}\right) & \operatorname{Cov}\left(\hat{\beta}_{k}, \hat{\beta}_{1}\right) & \cdots & \mathbb{D}\left(\hat{\beta}_{k}\right)
\end{array}\right)
$$

которая, оказывается, равна
 $$\sigma^{2}\left(\mathbf{X}^{T} \mathbf{X}\right)^{-1}$$

In [ ]:
def assessing_significance_variable(df,feautures_names, target, estimates_of_b, sigma_squared, index):
    """
    Функция для проведения теста на значимость коэффициента линейной регрессии.

    Параметры:
    df - весь дата фрейм задания
    feautures_names - признаки для оценивания
    target - признак для предсказывания
    estimates_of_b -- вектор оценок коэффициентов
    sigma_sq_hat -- оценка дисперсии ошибок
    index -- индекс коэффициента для теста на значимость

    Возвращает:
    p_value -- p-value для теста на значимость коэффициента
    """

    X = df[feautures_names].values
    Y = df[target].values

    X = np.hstack((np.ones((X.shape[0], 1)), X))

    cov_matrix = sigma_squared * np.linalg.inv(X.T @ X)

    dispersion_coef =  cov_matrix[index][index]

    standart_error_coef = np.sqrt(dispersion_coef)

    coef_st = estimates_of_b[index] / standart_error_coef

    p_value = 2 * (1 - norm.cdf(abs(coef_st)))

    return p_value



#### task4

Произведите отбор наиболее значимых переменных с помощью процедуры
backwards elimination:


1. Сначала обучите модель используя все доступные переменные;
2. Проведите тест на значимость для каждого коэффициента и найдите
соответствующее ему p–value;
3. Если все p–values оказались меньше 0.05, то алгоритм завершает работу. Иначе исключается переменная с наибольшим значением p–value,
ммодель обучается заново с нуля и мы повторяем алгоритм с шага 2.


In [ ]:
def backwards_elimination(df, target):
    remaining_features = df.drop(columns=target).columns.tolist()

    while True:
        estimates_of_b, estimates_of_y, sigma_squared = linear_regression(df,target, remaining_features)
        p_values = [assessing_significance_variable(df, remaining_features,target, estimates_of_b, sigma_squared, i)
                    for i in range(len(remaining_features))]
        print(p_values)
        if all(p < 0.05 for p in p_values):
            break

        max_p_index = p_values.index(max(p_values))
        removed_feature = remaining_features.pop(max_p_index)
    return remaining_features

In [ ]:
selected_features = backwards_elimination(df3, 'price')
print("Selected features:", selected_features)

 feautures: ['crime_rate', 'resid_area', 'air_qual', 'room_num', 'age', 'dist1', 'teachers', 'poor_prop', 'n_hos_beds', 'n_hot_rooms', 'rainfall', 'airport_YES', 'waterbody_River']
Оценка параметров линейной регрессии:
Intercept (β0): 12.435468772499462
Коэффициенты (β): [-5.10547675e-01 -2.66277810e+00 -2.42479750e+01  2.76132002e+00
  1.64990274e-02 -7.87140876e+00  1.77297734e+01 -1.05669926e+01
  2.40286065e-01  2.03149389e-02  7.62971914e-02  1.04735364e+00
 -1.83115753e-01]
Оценка дисперсии ошибок (σ̂²): 19.22036470200053
[0.3531525677991567, 0.09196484778936953, 0.2334913051005123, 3.1010354792693207e-05, 3.736344567073502e-12, 0.19081537623277822, 7.172040739078511e-14, 2.0736745653948674e-12, 0.0, 0.07634091495092576, 0.5872757721600994, 0.8948621492709488, 0.009555437263229827]
 feautures: ['crime_rate', 'resid_area', 'air_qual', 'room_num', 'age', 'dist1', 'teachers', 'poor_prop', 'n_hos_beds', 'n_hot_rooms', 'rainfall', 'waterbody_River']
Оценка параметров линейной регресси